In [1]:
from numpy import *
from time import *


def load_simple_data():
    """
    Function:
        创建加载简单数据集
    Parameters:
        无
    Returns:
         simple_data - 简单数据集
    Modify:
        2018-12-27
    """
    simple_data = [['r', 'z', 'h', 'j', 'p'],
                   ['z', 'y', 'x', 'w', 'v', 'u', 't', 's'],
                   ['z'],
                   ['r', 'x', 'n', 'o', 's'],
                   ['y', 'r', 'x', 'z', 'q', 't', 'p'],
                   ['y', 'z', 'x', 'e', 'q', 's', 't', 'm']]
    return simple_data


def create_init_set(data_set):
    """
    Function:
        计算项集在数据集的出现次数
    Parameters:
        data_set - 数据集
    Returns:
         ret_dict - 包含出现次数的项集字典
    Modify:
        2018-12-27
    """
    ret_dict = {}
    for trans in data_set:
        ret_dict[frozenset(trans)] = 1
    return ret_dict


def update_tree(items, in_tree, header_table, count):
    """
    Function:
        更新树节点，让FP树生长
    Parameters:
        items - 项集
        in_tree - 当前FP树
        header_table - 头指针表
        count - 次数
    Returns:
         无
    Modify:
        2018-12-27
    """
    # 判断排序后列表的第一个元素是否已经是根节点的子节点
    if items[0] in in_tree.children:
        # 添加出现次数
        # children = {}
        in_tree.children[items[0]].inc(count)
    else:
        # 创建根节点的子节点
        in_tree.children[items[0]] = TreeNode(items[0], count, in_tree)
        # 如果该元素的后继节点不存在，则直接加入。如果有后继节点，则遍历链表尾部将其加入
        if header_table[items[0]][1] == None:
            header_table[items[0]][1] = in_tree.children[items[0]]
        else:
            update_header(header_table[items[0]][1], in_tree.children[items[0]])
    # 列表元素长度大于1，递归调用更新FP树函数
    if len(items) > 1:
        update_tree(items[1::], in_tree.children[items[0]], header_table, count)


def update_header(node_to_test, target_node):
    """
    Function:
        更新头指针表的节点链接
    Parameters:
        node_to_test - 遍历节点
        target_node - 目标节点
    Returns:
         无
    Modify:
        2018-12-27
    """
    # 遍历到链表尾节点
    while (node_to_test.node_link != None):
        node_to_test = node_to_test.node_link
    # 将刚添加的树节点加入链表的尾部
    node_to_test.node_link = target_node


def create_tree(data_set, min_sup=1):
    """
    Function:
        遍历数据集两次构建FP树
    Parameters:
        data_set - 包含项集出现次数的数据集字典
        min_sup - 最小支持度
    Returns:
         ret_tree - FP树
         hearder_table - 头指针表
    Modify:
        2018-12-27
    """
    hearder_table = {}
    # 第一次遍历数据集，获取单个元素的次数
    for trans in data_set:
        for item in trans:
            hearder_table[item] = hearder_table.get(item, 0) + data_set[trans]
    # 去除不满足最小支持度的单个元素
    for k in list(hearder_table.keys()):
        if hearder_table[k] < min_sup:
            del (hearder_table[k])
    freq_item_set = set(hearder_table.keys())
    # 无频繁项就直接返回
    if len(freq_item_set) == 0:
        return None, None
    # 扩展头指针表，添加指向每种类型第一个元素的指针（节点链接）
    for k in hearder_table:
        hearder_table[k] = [hearder_table[k], None]
    # 创建根节点
    ret_tree = TreeNode('Null Set', 1, None)
    # 第二次遍历数据集，构建FP树
    for tran_set, count in data_set.items():
        # tran_set: frozenset({'h', 'p', 'z', 'j', 'r'})
        # count: 1
        local_d = {}
        # 如果单个元素是频繁项，则加入localD列表
        for item in tran_set:
            if item in freq_item_set:
                # hearder_table:{'b': [3, None]}
                local_d[item] = hearder_table[item][0]
        # localD: {'r': 3, 'j': 1, 'z': 5, 'h': 1, 'p': 2}
        if len(local_d) > 0:
            # 元素按出现次数排序
            ordered_items = [v[0] for v in sorted(local_d.items(), key=lambda p: p[1], reverse=True)]
            # 更新FP树，让FP树生长
            update_tree(ordered_items, ret_tree, hearder_table, count)
    return ret_tree, hearder_table


class TreeNode:
    # name：节点元素名称，在构造时初始化为给定值
    # count：出现次数，在构造时初始化为给定值
    # node_link：指向下一个相似节点的指针，默认为None
    # parent：指向父节点的指针，在构造时初始化为给定值
    # children：指向子节点的字典，以子节点的元素名称为键，指向子节点的指针为值，初始化为空字典
    def __init__(self, name_value, num_occur, parent_node):
        self.name = name_value
        self.count = num_occur
        self.node_link = None
        self.parent = parent_node
        self.children = {}

    def inc(self, num_occur):
        # 增加节点出现次数
        self.count += num_occur

    def disp(self, ind=1):
        # 用于将树以文本形式显示
        print('  ' * ind, self.name, ' ', self.count)
        for child in self.children.values():
            child.disp(ind + 1)


def ascend_tree(leaf_node, prefix_path):
    """
    Function:
        根据当前节点向前追溯至根节点，记录前缀路径
    Parameters:
        leaf_node - 给定元素项节点
        prefix_path - 前缀路径列表
    Returns:
         无
    Modify:
        2019-1-6
    """
    # 如果节点有父节点，则将当前节点添加至前缀路径中，之后再递归向上追溯
    if leaf_node.parent != None:
        prefix_path.append(leaf_node.name)
        ascend_tree(leaf_node.parent, prefix_path)


def find_prefix_path(base_pat, tree_node):
    """
    Function:
        发现以给定元素项结尾的所有前缀路径
    Parameters:
        base_pat - 元素项
        tree_node - 需遍历节点
    Returns:
         cond_pats - 所有条件模式基字典
    Modify:
        2019-1-6
    """
    # 所有条件模式基字典
    cond_pats = {}
    # 遍历该节点的整个链表节点，记录每个节点的前缀路径，并将其添加至条件模式基当中
    while tree_node != None:
        prefix_path = []
        ascend_tree(tree_node, prefix_path)
        # 因为该节点也被加进了路径当中，所以需要路径的长度大于1
        if len(prefix_path) > 1:
            # 如果有前缀路径，则将前缀路径加入条件模式基集合中，并且将该元素在该前缀路径中出现的次数也添加进去
            cond_pats[frozenset(prefix_path[1:])] = tree_node.count
        # 当前节点的条件模式基查找完毕后，继续查找头指针链表中下一个节点的条件模式基
        tree_node = tree_node.node_link
    return cond_pats


def mine_tree(in_tree, header_table, min_sup, pre_fix, freq_item_list):
    """
    Function:
        创建条件模式树
    Parameters:
        in_tree - FP树
        header_table - 头指针表
        min_sup - 最小支持度
        pre_fix - 上一次递归的频繁项集合的前缀
        freq_item_list - 频繁项集列表
    Returns:
        无
    Modify:
        2019-1-6
    """
    # 对头指针表中的元素项按照其出现频率从小到大进行排序
    big_l = [v[0] for v in sorted(header_table.items(), key=lambda p:p[1][0])]
    # 遍历单元素频繁集
    for base_pat in big_l:
        new_freq_set = pre_fix.copy()
        new_freq_set.add(base_pat)
        freq_item_list.append(new_freq_set)
        # 获得该元素的所有条件模式基，相当于一个事务集合
        cond_patt_bases = find_prefix_path(base_pat, header_table[base_pat][1])
        # 根据所有条件模式基集合来构建条件模式树
        my_cond_tree, my_head = create_tree(cond_patt_bases, min_sup)
        # 如果条件模式树的头指针表不空(每次建树时对元素支持度有要求
        # 如果小于支持度则该元素不参与建树过程，所以在建树时，条件模式基中的元素会越来越少，最后会是空树)，则递归建树
        if my_head != None:
            print('conditional tree for:', new_freq_set)
            my_cond_tree.disp(1)
            mine_tree(my_cond_tree, my_head, min_sup, new_freq_set, freq_item_list)


if __name__ == '__main__':
    # simple_data = load_simple_data()
    # init_data = create_init_set(simple_data)
    # print(simple_data)
    # print(init_data)
    '''
    simple_data = load_simple_data()
    init_data = create_init_set(simple_data)
    my_fp_tree, my_header_tab = create_tree(init_data, 3)
    my_fp_tree.disp()
    ''' 
    '''
    simple_data = load_simple_data()
    init_data = create_init_set(simple_data)
    my_fp_tree, my_header_tab = create_tree(init_data, 3)
    cond_pats_1 = find_prefix_path('x', my_header_tab['x'][1])
    print(cond_pats_1)
    cond_pats_2 = find_prefix_path('z', my_header_tab['z'][1])
    print(cond_pats_2)
    cond_pats_3 = find_prefix_path('r', my_header_tab['r'][1])
    print(cond_pats_3)
    '''
    simple_data = load_simple_data()
    init_data = create_init_set(simple_data)
    my_fp_tree, my_header_tab = create_tree(init_data, 3)
    freq_items = []
    mine_tree(my_fp_tree, my_header_tab, 3, set([]), freq_items)
    print(freq_items)
'''
    start_time = time()
    parse_dat = [line.split() for line in open('./machinelearninginaction/Ch12/kosarak.dat').readlines()]
    init_set = create_init_set(parse_dat)
    my_fp_tree, my_header_tab = create_tree(init_set, 100000)
    my_freq_list = []
    mine_tree(my_fp_tree, my_header_tab, 100000, set([]), my_freq_list)
    end_time = time()
    print('被10万或者更多人浏览过的新闻报道或报道集合数:', len(my_freq_list))
    print('被10万或者更多人浏览过的新闻报道或报道集合:', my_freq_list)
    print('总共执行时间:', end_time - start_time)
'''

conditional tree for: {'y'}
   Null Set   1
     z   3
       x   3
conditional tree for: {'y', 'x'}
   Null Set   1
     z   3
conditional tree for: {'s'}
   Null Set   1
     x   3
conditional tree for: {'t'}
   Null Set   1
     y   3
       z   3
         x   3
conditional tree for: {'z', 't'}
   Null Set   1
     y   3
conditional tree for: {'t', 'x'}
   Null Set   1
     z   3
       y   3
conditional tree for: {'y', 't', 'x'}
   Null Set   1
     z   3
conditional tree for: {'x'}
   Null Set   1
     z   3
[{'r'}, {'y'}, {'y', 'z'}, {'y', 'x'}, {'y', 'z', 'x'}, {'s'}, {'s', 'x'}, {'t'}, {'y', 't'}, {'z', 't'}, {'z', 'y', 't'}, {'t', 'x'}, {'z', 't', 'x'}, {'y', 't', 'x'}, {'y', 'z', 't', 'x'}, {'x'}, {'z', 'x'}, {'z'}]


"\n    start_time = time()\n    parse_dat = [line.split() for line in open('./machinelearninginaction/Ch12/kosarak.dat').readlines()]\n    init_set = create_init_set(parse_dat)\n    my_fp_tree, my_header_tab = create_tree(init_set, 100000)\n    my_freq_list = []\n    mine_tree(my_fp_tree, my_header_tab, 100000, set([]), my_freq_list)\n    end_time = time()\n    print('被10万或者更多人浏览过的新闻报道或报道集合数:', len(my_freq_list))\n    print('被10万或者更多人浏览过的新闻报道或报道集合:', my_freq_list)\n    print('总共执行时间:', end_time - start_time)\n"

In [2]:
import pprint
   
def loadDataSet():
    dataSet = [['bread', 'milk', 'vegetable', 'fruit', 'eggs'],
               ['noodle', 'beef', 'pork', 'water', 'socks', 'gloves', 'shoes', 'rice'],
               ['socks', 'gloves'],
               ['bread', 'milk', 'shoes', 'socks', 'eggs'],
               ['socks', 'shoes', 'sweater', 'cap', 'milk', 'vegetable', 'gloves'],
               ['eggs', 'bread', 'milk', 'fish', 'crab', 'shrimp', 'rice']]
   
    return dataSet
   
def transfer2FrozenDataSet(dataSet):
    frozenDataSet = {}
    for elem in dataSet:
        frozenDataSet[frozenset(elem)] = 1
   
    return frozenDataSet
   
class TreeNode:
    def __init__(self, nodeName, count, nodeParent):
        self.nodeName = nodeName
        self.count = count
        self.nodeParent = nodeParent
        self.nextSimilarItem = None
        self.children = {}
   
    def increaseC(self, count):
        self.count += count
   
   
def createFPTree(frozenDataSet, minSupport):
    # scan dataset at the first time, filter out items which are less than minSupport
    headPointTable = {}
    for items in frozenDataSet:
        for item in items:
            headPointTable[item] = headPointTable.get(item, 0) + frozenDataSet[items]
    headPointTable = {
        k: v
        for k, v in headPointTable.items() if v >= minSupport
    }
    frequentItems = set(headPointTable.keys())
    if len(frequentItems) == 0: return None, None
   
    for k in headPointTable:
        headPointTable[k] = [headPointTable[k], None]
   
    fptree = TreeNode("null", 1, None)
    # scan dataset at the second time, filter out items for each record
    for items, count in frozenDataSet.items():
        frequentItemsInRecord = {}
        for item in items:
            if item in frequentItems:
                frequentItemsInRecord[item] = headPointTable[item][0]
        if len(frequentItemsInRecord) > 0:
            frequentItemsInRecord = sorted(frequentItemsInRecord.items(), key=lambda v: v[0])
            orderedFrequentItems = [v[0] for v in sorted(frequentItemsInRecord, key=lambda v: v[1], reverse=True)]
            updateFPTree(fptree, orderedFrequentItems, headPointTable, count)
   
    return fptree, headPointTable
   
def updateFPTree(fptree, orderedFrequentItems, headPointTable, count):
    # handle the first item
    if orderedFrequentItems[0] in fptree.children:
        fptree.children[orderedFrequentItems[0]].increaseC(count)
    else:
        fptree.children[orderedFrequentItems[0]] = TreeNode(orderedFrequentItems[0], count, fptree)
   
        # update headPointTable
        if headPointTable[orderedFrequentItems[0]][1] == None:
            headPointTable[orderedFrequentItems[0]][1] = fptree.children[orderedFrequentItems[0]]
        else:
            updateHeadPointTable(headPointTable[orderedFrequentItems[0]][1], fptree.children[orderedFrequentItems[0]])
    # handle other items except the first item
    if (len(orderedFrequentItems) > 1):
        updateFPTree(fptree.children[orderedFrequentItems[0]], orderedFrequentItems[1::], headPointTable, count)
   
   
def updateHeadPointTable(headPointBeginNode, targetNode):
    while (headPointBeginNode.nextSimilarItem != None):
        headPointBeginNode = headPointBeginNode.nextSimilarItem
    headPointBeginNode.nextSimilarItem = targetNode
   
   
def mineFPTree(headPointTable, prefix, frequentPatterns, minSupport):
    # for each item in headPointTable, find conditional prefix path, create conditional fptree,
    # then iterate until there is only one element in conditional fptree
    headPointItems = [v[0] for v in sorted(headPointTable.items(), key=lambda v: v[1][0])]
    if (len(headPointItems) == 0): return
   
    for headPointItem in headPointItems:
        newPrefix = prefix.copy()
        newPrefix.add(headPointItem)
        support = headPointTable[headPointItem][0]
        frequentPatterns[frozenset(newPrefix)] = support
   
        prefixPath = getPrefixPath(headPointTable, headPointItem)
        if (prefixPath != {}):
            conditionalFPtree, conditionalHeadPointTable = createFPTree(prefixPath, minSupport)
            if conditionalHeadPointTable != None:
                mineFPTree(conditionalHeadPointTable, newPrefix, frequentPatterns, minSupport)
   
   
def getPrefixPath(headPointTable, headPointItem):
    prefixPath = {}
    beginNode = headPointTable[headPointItem][1]
    prefixs = ascendTree(beginNode)
    if ((prefixs != [])):
        prefixPath[frozenset(prefixs)] = beginNode.count
   
    while (beginNode.nextSimilarItem != None):
        beginNode = beginNode.nextSimilarItem
        prefixs = ascendTree(beginNode)
        if (prefixs != []):
            prefixPath[frozenset(prefixs)] = beginNode.count
   
    return prefixPath
   
def ascendTree(treeNode):
    prefixs = []
    while ((treeNode.nodeParent != None) and (treeNode.nodeParent.nodeName != 'null')):
        treeNode = treeNode.nodeParent
        prefixs.append(treeNode.nodeName)
   
    return prefixs
   
def rulesGenerator(frequentPatterns, minConf, rules):
    for frequentset in frequentPatterns:
        if (len(frequentset) > 1):
            getRules(frequentset, frequentset, rules, frequentPatterns, minConf)
   
   
def removeStr(set, str):
    tempSet = []
    for elem in set:
        if (elem != str):
            tempSet.append(elem)
    tempFrozenSet = frozenset(tempSet)
   
    return tempFrozenSet
   
def getRules(frequentset, currentset, rules, frequentPatterns, minConf):
    for frequentElem in currentset:
        subSet = removeStr(currentset, frequentElem)
        confidence = frequentPatterns[frequentset] / frequentPatterns[subSet]
        if (confidence >= minConf):
            flag = False
            for rule in rules:
                if (rule[0] == subSet and rule[1] == frequentset - subSet):
                    flag = True
   
            if (flag == False):
                rules.append((subSet, frequentset - subSet, confidence))
   
            if (len(subSet) >= 2):
                getRules(frequentset, subSet, rules, frequentPatterns, minConf)
   
   
if __name__ == '__main__':
    dataSet = loadDataSet()
    frozenDataSet = transfer2FrozenDataSet(dataSet)
    minSupport = 3
    fptree, headPointTable = createFPTree(frozenDataSet, minSupport)
    frequentPatterns = {}
    prefix = set([])
    mineFPTree(headPointTable, prefix, frequentPatterns, minSupport)
    print("frequent patterns:")
    pprint.pprint(frequentPatterns)
   
    minConf = 0
    rules = []
    rulesGenerator(frequentPatterns, minConf, rules)
    print("association rules:")
    pprint.pprint(rules)
    print('rules num:', len(rules))

frequent patterns:
{frozenset({'bread'}): 3,
 frozenset({'milk', 'bread'}): 3,
 frozenset({'eggs'}): 3,
 frozenset({'milk', 'eggs'}): 3,
 frozenset({'bread', 'eggs'}): 3,
 frozenset({'milk'}): 4,
 frozenset({'milk', 'bread', 'eggs'}): 3,
 frozenset({'gloves'}): 3,
 frozenset({'socks'}): 4,
 frozenset({'gloves', 'socks'}): 3,
 frozenset({'shoes'}): 3,
 frozenset({'socks', 'shoes'}): 3}
association rules:
[(frozenset({'bread'}), frozenset({'milk'}), 1.0),
 (frozenset({'milk'}), frozenset({'bread'}), 0.75),
 (frozenset({'eggs'}), frozenset({'milk'}), 1.0),
 (frozenset({'milk'}), frozenset({'eggs'}), 0.75),
 (frozenset({'bread', 'eggs'}), frozenset({'milk'}), 1.0),
 (frozenset({'eggs'}), frozenset({'milk', 'bread'}), 1.0),
 (frozenset({'bread'}), frozenset({'milk', 'eggs'}), 1.0),
 (frozenset({'milk', 'eggs'}), frozenset({'bread'}), 1.0),
 (frozenset({'milk'}), frozenset({'bread', 'eggs'}), 0.75),
 (frozenset({'milk', 'bread'}), frozenset({'eggs'}), 1.0),
 (frozenset({'eggs'}), frozenset({

In [38]:
for i in frequentPatterns:
    print(i)
    print(frequentPatterns[i])

frozenset({'bread'})
3
frozenset({'bread', 'milk'})
3
frozenset({'eggs'})
3
frozenset({'bread', 'eggs'})
3
frozenset({'milk', 'eggs'})
3
frozenset({'bread', 'milk', 'eggs'})
3
frozenset({'shoes'})
3
frozenset({'socks', 'shoes'})
3
frozenset({'gloves'})
3
frozenset({'socks', 'gloves'})
3
frozenset({'milk'})
4
frozenset({'socks'})
4
